In [1]:
import json
import random
import requests
import time
import csv
import codecs

In [2]:
"""
    爬取网易云音乐歌曲的精彩评论
"""

def start_spider(song_id):
    """ 评论数据采用 AJAX 技术获得, 下面才是获取评论的请求地址 """
    url = 'http://music.163.com/weapi/v1/resource/comments/R_SO_4_{}?csrf_token='.format(song_id)

    headers = {
        'User-agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 UBrowser/6.2.3964.2 Safari/537.36',
        'Origin': 'http://music.163.com',
        'Referer': 'http://music.163.com/song?id={}'.format(song_id),
    }

    formdata = {
        'params': '57Wh2mgebLOOPQVBc+B2wz4sCCH/nXZFEoTc/XNySiqT0V7ZxUADzDNgTXXhYgAJ5BNMryMgxhdwNzF1GyxDZo3iR9/YYbWgCAQHC5DCDuObqvxNcOcnQDaRqJCrqQcrEABW1SwKitfbD3wMEyB4tJu+rU8goSwg2FP/PBBLs9DVs1iWdWGjV6CdrocA36Rs',
        'encSecKey': '63774137ba4f5cc60d1b6a3bc14985a9563a7bfdec4f3e74297ffc07514adf18f90620933a01c2db4ca989cc4e1dfc49789981424c294a34e48c2cbe7aa51533a5cc5b5776a9e499cd08770bc596655dbe8e001d1ed5fd47a27dd195128480820cc67a799d341f95d447e3522851f2b64ad1cb8350e2015b265b9e684179351c',
    }

    response = requests.post(url, headers=headers, data=formdata)
    print('请求 [ ' + url + ' ], 状态码为 ',response.status_code)
    # get_hot_comments(response.text)
    # 将数据写到 CSV 文件中
    write_to_file(get_hot_comments(response.text))

In [3]:
def get_hot_comments(response):
    """ 获取精彩评论
    请求返回结果是 Json 数据格式, 使用 json.loads(response) 将其转化为字典类型, 就可以使用 key-value 形式获取值
    """
    data_list = []
    data = {}

    for comment in json.loads(response)['hotComments']:
        data['userId'] = comment['user']['userId']
        data['nickname'] = comment['user']['nickname']
        data['content'] = comment['content']
        data['likedCount'] = comment['likedCount']
        data_list.append(data)
        data = {}
    # print(data_list)
    return data_list

In [4]:
def write_to_file(datalist):
    print('开始将数据持久化……')
    file_name = 'netease_buzz.csv'

    with codecs.open(file_name, 'a+', 'utf-8') as csvfile:
        filednames = ['用户Id', '昵称', '评论内容', '点赞数']
        writer = csv.DictWriter(csvfile, fieldnames=filednames)

        writer.writeheader()
        for data in datalist:
            print(data)
            try:
                writer.writerow({filednames[0]: data['userId'],
                                 filednames[1]: data['nickname'],
                                 filednames[2]: data['content'],
                                 filednames[3]: data['likedCount']})
            except UnicodeEncodeError:
                print("编码错误, 该数据无法写到文件中, 直接忽略该数据")

    print('成功将数据写入到 ' + file_name + ' 中！')

In [5]:
def get_song_id(url):
    """ 从 url 中截取歌曲的 id """
    song_id = url.split('=')[1]
    return song_id

In [ ]:
def main():
    songs_url_list = [
        'http://music.163.com/#/song?id=186016',  # 晴天
        'http://music.163.com/#/song?id=186001',  # 七里香
        'http://music.163.com/#/song?id=27876900',  # Here We Are Again 《喜剧之王》电影插曲
        'http://music.163.com/#/song?id=439915614',  # 刚好遇见你
        'http://music.163.com/#/song?id=139774',  # The truth that you leave
        'http://music.163.com/#/song?id=29567189',  # 理想
        'http://music.163.com/#/song?id=308353',  # 钟无艳
        'http://music.163.com/#/song?id=31445772',  # 理想三旬
        'http://music.163.com/#/song?id=439915614',  # 刚好遇见你
        'http://music.163.com/#/song?id=28815250',  # 平凡之路
        'http://music.163.com/#/song?id=25706282',  # 夜空中最亮的星
        'http://music.163.com/#/song?id=436514312',  # 成都
    ]

    for each in songs_url_list:
        start_spider(get_song_id(each))
        time.sleep(random.randint(5, 8))


if __name__ == '__main__':
    main()

请求 [ http://music.163.com/weapi/v1/resource/comments/R_SO_4_186016?csrf_token= ], 状态码为  200
开始将数据持久化……
{'nickname': '蛋蛋是圆D', 'userId': 548345, 'likedCount': 710304, 'content': '高一听的，那时候遇到了孩儿他妈，然后就这么幸福下来了'}
{'nickname': '-荒灰黄-', 'userId': 16838326, 'likedCount': 342550, 'content': '老子要听一辈子周杰伦！！！'}
{'nickname': '全球单身狗反秀恩爱联盟荣誉会长', 'userId': 48794963, 'likedCount': 330432, 'content': '现在想来 我们这波第一批老去的90后还是挺幸运的 在我们最好的年龄遇到了最好的华语乐坛（周杰伦巅峰 林俊杰 SHE  潘玮柏 蔡依林…）遇到了巅峰的星爷 遇到了最好的西科东艾北卡南麦 动画城陪我们成长 周杰伦陪我们成熟 我们看着星爷老去 见证科比退役 或许我们不是最好的一代 但一定是最精彩的一代'}
{'nickname': '长在地上的姑娘', 'userId': 43108078, 'likedCount': 264096, 'content': '周杰伦 你要对那么多人的青春负责'}
{'nickname': '我竟然是一个柚子', 'userId': 46299149, 'likedCount': 223023, 'content': '初中听周杰伦被同学嘲笑：唱的什么鬼…长那么丑…词都唱不清…他们哪知道若干年后会听着他的歌流泪一整晚，原来我们输给了时光，败给了唇红齿白的自己😭'}
{'nickname': '小東得100分', 'userId': 116973683, 'likedCount': 191466, 'content': '2004年，高一，男班长，你很喜欢周杰伦，我存了很久的钱在你生日那天买了一张周杰伦的专辑送给你。2016年，如今，你女儿都可以上幼儿园了吧，但是你一直不知道我曾经也暗恋过你。是的，我只能暗恋，因为我也是男的。'}
{'nickname': '哥特式的永恒', 'userI